In [1]:
# -*- coding: UTF-8 -*-
import json 
imgs = json.load(open('/data/common/data/ai_challenger_caption/ai_challenger_caption_train_20170902/caption_train_annotations_20170902.json', 'r'))

In [10]:
bd = '、'
sent_lengths = {}
for i in range (2):
    for j in range(5):
        example = imgs[i]['caption'][j].replace(bd.decode('UTF-8'), '')
        seg_list = jieba.cut(example, cut_all=False)
        sent_lengths[nw] = sent_lengths.get(nw, 0) + 1
        print example

两个衣着休闲的人在平整的道路上交谈
一个穿着红色上衣的男人和一个穿着灰色裤子的男人站在室外的道路上交谈
室外的公园里有两个穿着长裤的男人在交流
街道上有一个穿着深色外套的男人和一个穿着红色外套的男人在交谈
道路上有一个身穿红色上衣的男人在和一个抬着左手的人讲话
房间里有三个坐在桌子旁的人在吃饭
两个戴着帽子的人和一个短发男人坐在房间里就餐
房间里有一个男人和两个戴帽子的老人在吃饭
室内三个衣着各异的人坐在桌子旁交谈
屋子里有三个衣着各异的人坐在椅子上


## 对caption进行分词并保存各词数量和句子长度

In [3]:
import jieba
import time
bd = '、'
counts = {}
sent_lengths = {}
i = 0
start = time.clock()
for i in range(210000):
    for j in range(5):
        example = imgs[i]['caption'][j].replace(bd.decode('UTF-8'), '')
        seg_list = jieba.cut(example, cut_all=False)
        nw = 0
        for w in seg_list:
            counts[w]=counts.get(w, 0) + 1
            nw = nw + 1
        sent_lengths[nw] = sent_lengths.get(nw, 0) + 1
cw = sorted([(count,w) for w,count in counts.items()], reverse=True)
max_len = max(sent_lengths.keys()) 
elapsed = (time.clock() - start)
print("CUT time used:",elapsed)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.322 seconds.
Prefix dict has been built succesfully.


('Time used:', 150.809755)


## 设置阈值,查看词汇统计

In [20]:
# 设置词汇出现次数的阈值 查看词典中有多少词汇
count_thr = 3
print('top words and their counts:')
for i in range(20):
    print cw[i][1]
print('\n'.join(map(str,cw[:20])))
total_words = sum(counts.values())
print('total words:', total_words)
bad_words = [w for w,n in counts.items() if n <= count_thr]
vocab = [w for w,n in counts.items() if n > count_thr]
bad_count = sum(counts[w] for w in bad_words)
print len(counts)
print('number of bad words: %d/%d = %.2f%%' % (len(bad_words), len(counts), len(bad_words)*100.0/len(counts)))
print('number of words in vocab would be %d' % (len(vocab), ))
print('number of UNKs: %d/%d = %.2f%%' % (bad_count, total_words, bad_count*100.0/total_words))

# lets now produce the final annotations
if bad_count > 0:
# additional special UNK token we will use below to map infrequent words to
    print('inserting the special UNK token')
    vocab.append('UNK')

top words and their counts:
的
一个
在
上
男人
着
穿着
有
女人
两个
人
拿
站
里
球场上
右手
戴着
走
双手
道路
(1729421, u'\u7684')
(986863, u'\u4e00\u4e2a')
(862029, u'\u5728')
(582124, u'\u4e0a')
(554635, u'\u7537\u4eba')
(519252, u'\u7740')
(440255, u'\u7a7f\u7740')
(414516, u'\u6709')
(411376, u'\u5973\u4eba')
(271990, u'\u4e24\u4e2a')
(234585, u'\u4eba')
(228711, u'\u62ff')
(208127, u'\u7ad9')
(176569, u'\u91cc')
(142295, u'\u7403\u573a\u4e0a')
(138792, u'\u53f3\u624b')
(137708, u'\u6234\u7740')
(128076, u'\u8d70')
(127096, u'\u53cc\u624b')
(110284, u'\u9053\u8def')
('total words:', 13825829)
17652
number of bad words: 9084/17652 = 51.46%
number of words in vocab would be 8568
number of UNKs: 13654/13825829 = 0.10%
inserting the special UNK token


## 查看句子长度统计

In [65]:
print('max length sentence in raw data: ', max_len)
print('sentence length distribution 累计(count, number of words):')
sum_len = sum(sent_lengths.values())
perc = 0
for i in range(max_len+1):
    perc = perc + sent_lengths.get(i,0)*100.0/sum_len
    print('%2d: %10d   %f%%  %f' % (i, sent_lengths.get(i,0), sent_lengths.get(i,0)*100.0/sum_len,perc))
    

('max length sentence in raw data: ', 32)
sentence length distribution 累计(count, number of words):
 0:          0   0.000000%  0.000000
 1:          0   0.000000%  0.000000
 2:          0   0.000000%  0.000000
 3:          0   0.000000%  0.000000
 4:          0   0.000000%  0.000000
 5:          3   0.000286%  0.000286
 6:         16   0.001524%  0.001810
 7:        198   0.018857%  0.020667
 8:      17306   1.648190%  1.668857
 9:      70067   6.673048%  8.341905
10:     125226   11.926286%  20.268190
11:     173391   16.513429%  36.781619
12:     148898   14.180762%  50.962381
13:     118381   11.274381%  62.236762
14:      91879   8.750381%  70.987143
15:      70236   6.689143%  77.676286
16:      60209   5.734190%  83.410476
17:      52858   5.034095%  88.444571
18:      42799   4.076095%  92.520667
19:      31340   2.984762%  95.505429
20:      20143   1.918381%  97.423810
21:      12140   1.156190%  98.580000
22:       6826   0.650095%  99.230095
23:       3819   0.363714%  99.59

In [16]:

# if bad_count > 0:
#     print('inserting the special UNK token')
#     vocab.append('UNK')
start = time.clock()    
for i in range(210000):
    imgs[i]['final_captions'] = []
    for j in range(5):
        example = imgs[i]['caption'][j].replace(bd.decode('UTF-8'), '')
        seg_list = jieba.cut(example, cut_all=False)
        caption = [w if counts.get(w,0) > count_thr else 'UNK' for w in seg_list]
        imgs[i]['final_captions'].append(caption)
elapsed = (time.clock() - start)
print("Final captions time used:",elapsed)        

In [26]:
itow = {i+1:w for i,w in enumerate(vocab)} # a 1-indexed vocab translation table

wtoi = {w:i+1 for i,w in enumerate(vocab)} # inverse table
print wtoi.get('UNK')

8569


In [27]:
import numpy as np
max_length = 22
N = len(imgs)
M = sum(len(img['final_captions']) for img in imgs) # total number of captions
print N
print M

210000
1050000


In [28]:
print imgs[0]['final_captions']

[[u'\u4e24\u4e2a', u'\u8863\u7740', u'\u4f11\u95f2', u'\u7684', u'\u4eba', u'\u5728', u'\u5e73\u6574', u'\u7684', u'\u9053\u8def', u'\u4e0a', u'\u4ea4\u8c08'], [u'\u4e00\u4e2a', u'\u7a7f\u7740', u'\u7ea2\u8272', u'\u4e0a\u8863', u'\u7684', u'\u7537\u4eba', u'\u548c', u'\u4e00\u4e2a', u'\u7a7f\u7740', u'\u7070\u8272', u'\u88e4\u5b50', u'\u7684', u'\u7537\u4eba', u'\u7ad9', u'\u5728', u'\u5ba4\u5916', u'\u7684', u'\u9053\u8def', u'\u4e0a', u'\u4ea4\u8c08'], [u'\u5ba4\u5916', u'\u7684', u'\u516c\u56ed', u'\u91cc', u'\u6709', u'\u4e24\u4e2a', u'\u7a7f\u7740', u'\u957f\u88e4', u'\u7684', u'\u7537\u4eba', u'\u5728', u'\u4ea4\u6d41'], [u'\u8857\u9053', u'\u4e0a', u'\u6709', u'\u4e00\u4e2a', u'\u7a7f\u7740', u'\u6df1\u8272', u'\u5916\u5957', u'\u7684', u'\u7537\u4eba', u'\u548c', u'\u4e00\u4e2a', u'\u7a7f\u7740', u'\u7ea2\u8272', u'\u5916\u5957', u'\u7684', u'\u7537\u4eba', u'\u5728', u'\u4ea4\u8c08'], [u'\u9053\u8def', u'\u4e0a', u'\u6709', u'\u4e00\u4e2a', u'\u8eab\u7a7f', u'\u7ea2\u8272', u

In [29]:
start = time.clock() 
label_arrays = []
label_start_ix = np.zeros(N, dtype='uint32') # note: these will be one-indexed
label_end_ix = np.zeros(N, dtype='uint32')
label_length = np.zeros(M, dtype='uint32')
caption_counter = 0
counter = 1
for i,img in enumerate(imgs):
    n = len(img['final_captions']) 
    assert n > 0, 'error: some image has no captions'
    Li = np.zeros((n, max_length), dtype='uint32')
    for j,s in enumerate(img['final_captions']):
        label_length[caption_counter] = min(max_length, len(s)) # record the length of this sequence
        caption_counter += 1
        for k,w in enumerate(s):
            if k < max_length:
                Li[j,k] = wtoi[w]
    
    label_arrays.append(Li)
    label_start_ix[i] = counter
    label_end_ix[i] = counter + n - 1
    
    counter += n
elapsed = (time.clock() - start)
print("Word2index time used:",elapsed)    

In [30]:
L = np.concatenate(label_arrays, axis=0) # put all the labels together
assert L.shape[0] == M, 'lengths don\'t match? that\'s weird'
assert np.all(label_length > 0), 'error: some caption had no words?'

print('encoded captions to array of size ', L.shape)
#return L, label_start_ix, label_end_ix, label_length

('encoded captions to array of size ', (1050000, 22))


In [33]:
# create output h5 file
import h5py
N = len(imgs)
f_lb = h5py.File('../data/ImageCaption'+'_label.h5', "w")
f_lb.create_dataset("labels", dtype='uint32', data=L)
f_lb.create_dataset("label_start_ix", dtype='uint32', data=label_start_ix)
f_lb.create_dataset("label_end_ix", dtype='uint32', data=label_end_ix)
f_lb.create_dataset("label_length", dtype='uint32', data=label_length)
f_lb.close()

In [50]:
# create output json file
import os
out = {}
out['ix_to_word'] = itow # encode the (1-indexed) vocab
out['images'] = []
for i,img in enumerate(imgs):

    jimg = {}
    jimg['split'] = u'train'#img['split']
    #if 'filename' in img: jimg['file_path'] = os.path.join(img['filepath'], img['filename']) # copy it over, might need
    if 'image_id' in img: jimg['file_path'] = os.path.join(u'caption_train_images_20170902', img['image_id']) # copy it over, might need
    if 'image_id' in img: jimg['id'] = img['image_id']
    #if 'cocoid' in img: jimg['id'] = img['cocoid'] # copy over & mantain an id, if present (e.g. coco ids, useful)

    out['images'].append(jimg)

json.dump(out, open('../data/ImageCaption.json', 'w'))
print('wrote ', 'ImageCaption.json')

('wrote ', 'ImageCaption.json')


In [47]:
print out['images'][0]

{'split': u'train', 'file_path': u'caption_train_images_20170902/8f00f3d0f1008e085ab660e70dffced16a8259f6.jpg'}


In [100]:
coco_imgs = json.load(open('data/dataset_coco.json', 'r'))
coco_imgs = coco_imgs['images']

In [101]:
for i,img in enumerate(coco_imgs):
    print img
    if i == 1:
        break
    if 'image_id' in img: jimg['file_path'] = os.path.join(u'caption_train_images_20170902', img['image_id']) # copy it over, might need
#     jimg = {}
#     jimg['split'] = img['split']
#     if 'filename' in img: jimg['file_path'] = os.path.join(img['filepath'], img['filename']) # copy it over, might need
#     if 'cocoid' in img: jimg['id'] = img['cocoid'] # copy over & mantain an id, if present (e.g. coco ids, useful)

# out['images'].append(jimg)

json.dump(out, open(params['output_json'], 'w'))
print('wrote ', params['output_json'])

{u'sentids': [770337, 771687, 772707, 776154, 781998], u'filepath': u'val2014', u'filename': u'COCO_val2014_000000391895.jpg', u'imgid': 0, u'split': u'test', u'sentences': [{u'tokens': [u'a', u'man', u'with', u'a', u'red', u'helmet', u'on', u'a', u'small', u'moped', u'on', u'a', u'dirt', u'road'], u'raw': u'A man with a red helmet on a small moped on a dirt road. ', u'imgid': 0, u'sentid': 770337}, {u'tokens': [u'man', u'riding', u'a', u'motor', u'bike', u'on', u'a', u'dirt', u'road', u'on', u'the', u'countryside'], u'raw': u'Man riding a motor bike on a dirt road on the countryside.', u'imgid': 0, u'sentid': 771687}, {u'tokens': [u'a', u'man', u'riding', u'on', u'the', u'back', u'of', u'a', u'motorcycle'], u'raw': u'A man riding on the back of a motorcycle.', u'imgid': 0, u'sentid': 772707}, {u'tokens': [u'a', u'dirt', u'path', u'with', u'a', u'young', u'person', u'on', u'a', u'motor', u'bike', u'rests', u'to', u'the', u'foreground', u'of', u'a', u'verdant', u'area', u'with', u'a', u

In [40]:
[ref_words = []
sent=['1','2','3']
tmp_tokens = sent + ['<eos>']
print tmp_tokens
ref_words.append(' '.join(tmp_tokens))
print ref_words
for k in xrange(1,5):
    print k]

['1', '2', '3', '<eos>']
['1 2 3 <eos>']
1
2
3
4


In [49]:
print imgs[0]

{u'url': u'http://m4.biz.itc.cn/pic/new/n/71/65/Img8296571_n.jpg', u'image_id': u'8f00f3d0f1008e085ab660e70dffced16a8259f6.jpg', 'final_captions': [[u'\u4e24\u4e2a', u'\u8863\u7740', u'\u4f11\u95f2', u'\u7684', u'\u4eba', u'\u5728', u'\u5e73\u6574', u'\u7684', u'\u9053\u8def', u'\u4e0a', u'\u4ea4\u8c08'], [u'\u4e00\u4e2a', u'\u7a7f\u7740', u'\u7ea2\u8272', u'\u4e0a\u8863', u'\u7684', u'\u7537\u4eba', u'\u548c', u'\u4e00\u4e2a', u'\u7a7f\u7740', u'\u7070\u8272', u'\u88e4\u5b50', u'\u7684', u'\u7537\u4eba', u'\u7ad9', u'\u5728', u'\u5ba4\u5916', u'\u7684', u'\u9053\u8def', u'\u4e0a', u'\u4ea4\u8c08'], [u'\u5ba4\u5916', u'\u7684', u'\u516c\u56ed', u'\u91cc', u'\u6709', u'\u4e24\u4e2a', u'\u7a7f\u7740', u'\u957f\u88e4', u'\u7684', u'\u7537\u4eba', u'\u5728', u'\u4ea4\u6d41'], [u'\u8857\u9053', u'\u4e0a', u'\u6709', u'\u4e00\u4e2a', u'\u7a7f\u7740', u'\u6df1\u8272', u'\u5916\u5957', u'\u7684', u'\u7537\u4eba', u'\u548c', u'\u4e00\u4e2a', u'\u7a7f\u7740', u'\u7ea2\u8272', u'\u5916\u5957', u'